In [17]:
import pandas as pd
import numpy as np

import time

import torch
# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

#import warnings
#warnings.filterwarnings("ignore")

from transformers import BartForSequenceClassification, BartTokenizer, BartConfig

# internal libraries
from ressources import target_to_label, raw_target

# 

In [18]:
config = BartConfig.from_pretrained("valhalla/distilbart-mnli-12-9")
tokenizer = BartTokenizer.from_pretrained('valhalla/distilbart-mnli-12-9')
model = BartForSequenceClassification.from_pretrained('valhalla/distilbart-mnli-12-9')

In [19]:
data = pd.read_csv("osdg-data.csv")


df = data[(data['sdg'] == 12) & (data['label_osdg'] =="accepted")]


text = df["text"].iloc[0]
print(text)

This makes tourism a co-ordination-intensive, as well as information-intensive, industry (Zhang et al. The key components of tourism are accommodation, transport, attractions and excursions, and restaurants; all are ‘perishable’. This means that airline seats, hotel rooms and daily ticket sales, for example, cannot be stored for potential future sales. This level of uncertainty, coupled with the uncertainty of global trends and exogenous shocks, has become an important area of tourism supply chain research. Areas of particular interest include demand forecasting, yield or revenue management and inventory management (Zhang et al. Finally, the supply chains in tourism that already exist are usually part of the wider global operations of major hotels and resorts (for example Hilton and Four Seasons hotels) and of cruise ship operators (for example Carnival Corporation and Royal Caribbean).


In [20]:
def predict(premise,hypothesis) :    
    # run through model pre-trained on MNLI
    input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
    logits = model(input_ids)[0]

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true 
    entail_contradiction_logits = logits[:,[0,2]]
    # print(entail_contradiction_logits)
    probs = entail_contradiction_logits.softmax(dim=1)
    true_prob = probs[:,1].item() * 100
    print(f'Probability that {hypothesis} is true: {true_prob:0.2f}%')
    

In [21]:
labels = list(target_to_label.values())
labels_raw = list(raw_target.values())

for label in labels_raw :
    # Build hypothesis
    hypothesis = label
    
    start_time = time.time()
    predict(text,hypothesis)
    total_time = time.time() - start_time
    
    print(f"Total prediction time : {total_time:0.2f}s")
    

Probability that Implement the 10-year framework of programmes on sustainable consumption and production, all countries taking action, with developed countries taking the lead, taking into account the development and capabilities of developing countries is true: 43.10%
Total prediction time : 0.70s
Probability that By 2030, achieve the sustainable management and efficient use of natural resources is true: 13.91%
Total prediction time : 0.65s
Probability that By 2030, halve per capita global food waste at the retail and consumer levels and reduce food losses along production and supply chains, including post-harvest losses is true: 6.74%
Total prediction time : 0.63s
Probability that By 2020, achieve the environmentally sound management of chemicals and all wastes throughout their life cycle, in accordance with agreed international frameworks, and significantly reduce their release to air, water and soil in order to minimize their adverse impacts on human health and the environment is t